### doc_topics
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E TÓPICOS NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.

In [3]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from gensim import corpora, models, similarities #Latent Dirichlet Allocation implementation with Gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import clear_output

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

In [4]:
outputs = "../outputs/"

In [5]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Seleciona textos da base mysql para fazer modelagem de tópicos
NOTA: DEIXEI EM RAW PORQUE ESSA EXTRAÇÃO TORNA O CÓDIGO MUITO LENTO, MAS DEVE SER FEITO DESSA FORMA NO CPU DO RENATO
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
texts = []
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()

with SSHTunnelForwarder(('200.20.164.147', 22),
                        #ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        #ssh_username="marcelobribeiro",
                        ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           #user='marcelobribeiro', 
                           user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    captura documentos da tabela docs para posterior modelagem de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    cur.execute("SELECT * FROM CPDOC_AS.docs WHERE main_language = 1") # seleciona apenas textos majoritariamente em português    
    data = cur.fetchall()
    numrows = cur.rowcount
    percentil = numrows/100
    
    for row in data:
        count += 1
        if row is None: break

        if count % percentil == 0: 
            clear_output()
            print(int(count/percentil),'% done')

        text =  row[1]
        text = text.split()
        symbols = [x for x in string.punctuation]
        text = [p for p in text if p not in symbols]
        text = [p.strip(string.punctuation) for p in text]
        text = [p for p in text if not p.isdigit()]
        text = [p for p in text if len(p)>1]
        texts.append(text)
        
        #if count == 3: break #amostra de apenas 3 linhas. No computador do Renato, não usar o break.

········
········


In [15]:
# load nltk's English stopwords as variable called 'stopwords'

additional_words = ['mr','one', 'two', 'three', 'four', 
                    'five', 'um', 'dois', 'três', 'quatro', 
                    'cinco', 'janeiro', 'fevereiro', 'março', 
                    'abril', 'maio', 'junho', 'julho', 'agosto', 
                    'setembro', 'outubro', 'novembro', 'dezembro', 
                    'january', 'february', 'march', 'april', 'may', 
                    'june', 'july', 'august', 'september', 
                    'october', 'november', 'december', 'países', 
                    'ser', 'país', 'ainda', 'milhões', 'maior', 
                    'anos', 'grande', 'apenas', 'outros', 'pode', 
                    'parte', 'partes', 'item', 'vossa', 'nota', 
                    'havia', 'pt', 'vg', 'ptvg', 'eh', 'nr', 'hrs', 
                    'pais', 'parte', 'hoje', 'brasemb', 'ontem', 
                    'dia', 'countries', 'would', 'new', 'also', 
                    'must', 'draft', 'shall', 'item', 'page', 
                    'th', 'anos', 'ii', 'dias', 'poderá', 'caso', 
                    'casos', 'qualquer', 'ano', 'mil', 'pessoas', 
                    'único', 'única', 'únicos', 'únicas', 'índice', 
                    'expedido', 'co', 'mm', 'er', 'via', 'ww', 'ra', 
                    'ia', 'ca', 'nu', 'wa', 'aa', 'ms', 'dc', 'mmm', 'pa']

stopwords = nltk.corpus.stopwords.words('english') + \
            nltk.corpus.stopwords.words('portuguese') + \
            nltk.corpus.stopwords.words('french') + \
            nltk.corpus.stopwords.words('spanish') + \
            additional_words

stopwords = list(set(stopwords))

### Latent Dirichlet Allocation

In [16]:
#%time texts = [[word for word in text] for text in texts]
%time texts = [[word for word in text if word not in stopwords] for text in texts]

CPU times: user 1min 22s, sys: 0 ns, total: 1min 22s
Wall time: 1min 22s


In [17]:
#print(len([word for word in texts[0] if word not in stopwords]))
print(len(texts[0]))

125


In [18]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
#print(dictionary.token2id['']) #examina o id a partir do token
#print(dictionary.id2token[0]) #examina o token a partir do id
dictionary.filter_tokens(bad_ids=[0,]) #retira palavras a partir do id
corpus = [dictionary.doc2bow(text) for text in texts]

In [19]:
len(corpus)

9083

In [20]:
%time lda = models.LdaModel(corpus, num_topics=50, id2word=dictionary, passes=10)

CPU times: user 1h 9min 32s, sys: 5h 33min 34s, total: 6h 43min 7s
Wall time: 23min 24s


In [21]:
lda.print_topics(-1, num_words=20)

[(0,
  '0.177*petróleo + 0.072*saudita + 0.049*iraque + 0.039*arábia + 0.036*irã + 0.031*opep + 0.025*petrobras + 0.025*coveite + 0.021*petrobrás + 0.016*teerã + 0.015*iran + 0.013*barris + 0.012*arabia + 0.011*companhias + 0.010*pemex + 0.010*iraniano + 0.009*iraniana + 0.008*nioc + 0.008*compra + 0.007*oil'),
 (1,
  '0.027*acordo + 0.021*governo + 0.019*militar + 0.013*segurança + 0.010*material + 0.008*assistência + 0.007*república + 0.007*marinha + 0.007*forças + 0.007*nacional + 0.007*ministério + 0.007*presente + 0.006*brasil + 0.006*motivos + 0.006*governos + 0.006*excelência + 0.006*exército + 0.006*armadas + 0.006*militares + 0.006*unidos'),
 (2,
  '0.072*der + 0.042*die + 0.039*und + 0.018*sie + 0.017*den + 0.016*zu + 0.015*auf + 0.013*von + 0.009*issct + 0.009*ich + 0.009*beide + 0.008*alemanha + 0.008*federal + 0.008*mit + 0.007*brasilien + 0.007*zwischen + 0.006*im + 0.006*dem + 0.006*fúr + 0.005*xvi'),
 (3,
  '0.019*governo + 0.013*argentina + 0.011*brasileiro + 0.010*tra

In [22]:
lda.show_topics(num_topics=-1)

[(0,
  '0.177*petróleo + 0.072*saudita + 0.049*iraque + 0.039*arábia + 0.036*irã + 0.031*opep + 0.025*petrobras + 0.025*coveite + 0.021*petrobrás + 0.016*teerã'),
 (1,
  '0.027*acordo + 0.021*governo + 0.019*militar + 0.013*segurança + 0.010*material + 0.008*assistência + 0.007*república + 0.007*marinha + 0.007*forças + 0.007*nacional'),
 (2,
  '0.072*der + 0.042*die + 0.039*und + 0.018*sie + 0.017*den + 0.016*zu + 0.015*auf + 0.013*von + 0.009*issct + 0.009*ich'),
 (3,
  '0.019*governo + 0.013*argentina + 0.011*brasileiro + 0.010*tratado + 0.010*brasil + 0.010*relações + 0.008*reunião + 0.008*exteriores + 0.007*respeito + 0.007*assunto'),
 (4,
  '0.036*transporte + 0.016*terrestre + 0.013*dg + 0.011*transito + 0.010*freteiros + 0.010*tráfego + 0.008*trânsito + 0.008*empresas + 0.007*justa + 0.007*dner'),
 (5,
  '0.017*ministro + 0.016*silveira + 0.011*rio + 0.010*embaixador + 0.010*senhor + 0.010*azeredo + 0.009*antonio + 0.009*exteriores + 0.007*sr + 0.007*relações'),
 (6,
  '0.064*s

In [23]:
topics_matrix = lda.show_topics(formatted=False, num_words=20, num_topics=-1)
topics_matrix[34]

(34,
 [('pesca', 0.06967917974408365),
  ('eanes', 0.04026646214306908),
  ('barcos', 0.035897413195945263),
  ('ramalho', 0.025074744993800652),
  ('station', 0.013364091241367046),
  ('camarão', 0.0095153528731351713),
  ('fishing', 0.0090813570673366917),
  ('embarcação', 0.0087996034577154988),
  ('cpdoc', 0.0082752626064098288),
  ('sudepe', 0.0079392110427844893),
  ('shrimp', 0.0070353279362189271),
  ('embarcações', 0.0068513889989136019),
  ('vessels', 0.0062795848039468348),
  ('código', 0.0061986576493448251),
  ('vessel', 0.0061907048161978055),
  ('aas', 0.0061263651481450326),
  ('mre/vp', 0.0058456379354161585),
  ('territorial', 0.0057711963380910041),
  ('18/8/76', 0.0053890408281960112),
  ('religiosas', 0.0053588742977579967)])

Gerando a distribuição de tópicos para um novo documento:

#### Gera a distribuição de tópicos para um novo documento
Alimenta a base mysql com dados de score (relação) entre tópicos e documentos

In [ ]:
count = 0
#percentil = int(len(cursor.rowcount/100)

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        #ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        #ssh_username="marcelobribeiro",
                        ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           #user='marcelobribeiro', 
                           user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE IF EXISTS topic_doc_lang_filter")
    cur.execute('''CREATE TABLE IF NOT EXISTS topic_doc_lang_filter 
               (doc_id VARCHAR(31), topic_id smallint(6), topic_score FLOAT
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

    cur.execute("SELECT id,body FROM CPDOC_AS.docs")
    data = cur.fetchall()
    for row in data:
        count += 1 # for break event
        if row is None: break
        text =  row[1]
        text = text.split()
        text_bow = dictionary.doc2bow(text)
        #print(lda[text_bow])
        score_list = lda[text_bow]
        doc_id = row[0]
        for score in score_list:
            topic_id = str(score[0])
            topic_score = str(score[1])
            query = "INSERT INTO topic_doc_lang_filter VALUES (%s, %s, %s)"
            cur.execute(query, (doc_id, topic_id, topic_score))
        #print(doc_id, topic_id, topic_score)   
        cur.execute("ALTER TABLE CPDOC_AS.topic_doc_lang_filter ORDER BY topic_id ASC, topic_score DESC")
        
        #if count == 10: break #amostra de apenas 10 linhas

········
········


#### Visualizando os tópicos com o PyLDAvis

In [ ]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_langfilter_output.html'))